In [8]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager
import re
from collections import defaultdict
import time
import json

In [13]:
# executable_path = {'executable_path': ChromeDriverManager().install()}
executable_path = {'executable_path': '/Users/Justin1/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver'} 
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
url = 'https://transcripts.foreverdreaming.org/viewforum.php?f=574'
browser.visit(url)

In [ ]:
html = browser.html
bsoup = soup(html, 'html.parser')
transcript = {}
characters = []
script_df = pd.DataFrame(columns = ['character','line','season','episode','line_ID'])

for i in range(1,9):
    if i == 1:
        seas_ep = bsoup.find_all('h3')
        for j in range(1,(len(seas_ep))):
            link_text = seas_ep[j].a.get_text()
            if link_text.find('99') != -1:
                print("Found '99'...Moving to next line.")
                continue
            else:
                browser.links.find_by_partial_text(link_text).click()
                html = browser.html
                bsoup = soup(html, 'html.parser')
                scripts = bsoup.find('div', class_='postbody')
                all_lines = scripts.find_all('p', recursive=False)
                print(f'gathering scripts from "{link_text}"')
                for line in all_lines:
                    substring_list = ['Deleted','Season','Webisode']
                    try:
                        name = line.find('strong').get_text()
                        if any(substring in name for substring in substring_list) == True:
                            continue
                        else:
                            line = line.get_text().replace(f'{name}: ','')
                            if name not in characters:
                                characters.append(name)
                                seas=(int(link_text[:2]))*100
                                ep=int(link_text[3:5])
                                lineID= int((seas)+(ep))
                                temp_df = pd.DataFrame([[name,line,seas,ep,lineID]],
                                                       columns =['character','line','season','episode','line_ID'] )
                                script_df = pd.concat([script_df,temp_df], ignore_index=True, axis=0)
                            else:
                                seas=(int(link_text[:2]))*100
                                ep=int(link_text[3:5])
                                lineID= int((seas)+(ep))
                                temp_df = pd.DataFrame([[name,line,seas,ep,lineID]],
                                                       columns =['character','line','season','episode','line_ID'] )
                                script_df = pd.concat([script_df,temp_df], ignore_index=True, axis=0)
                    except:
                        print('SKIPPPPPPPPPPPPPED')
                        pass
                browser.back()

    else:
        page_numbers = browser.links.find_by_text(i)
        page_numbers.click()
        html = browser.html
        bsoup = soup(html, 'html.parser')
    #   time.sleep(3)
        seas_ep = bsoup.find_all('h3')
        for j in range(1,(len(seas_ep))):
            link_text = seas_ep[j].a.get_text()
            if link_text.find('99') != -1:
                print("Found '99'...Moving to next line.")
                continue
            else:
                browser.links.find_by_partial_text(link_text).click()
                html = browser.html
                bsoup = soup(html, 'html.parser')
                scripts = bsoup.find('div', class_='postbody')
                all_lines = scripts.find_all('p', recursive=False)
                print(f'gathering scripts from "{link_text}"')
                for line in all_lines:
                    substring_list = ['Deleted','Season','Webisode']
                    try:
                        name = line.find('strong').get_text()
                        if any(substring in name for substring in substring_list) == True:
                            continue
                        else:
                            line = line.get_text().replace(f'{name}: ','')
                            if name not in characters:
                                characters.append(name)
                                seas=(int(link_text[:2]))*100
                                ep=int(link_text[3:5])
                                lineID= int((seas)+(ep))
                                temp_df = pd.DataFrame([[name,line,seas,ep,lineID]],
                                                       columns =['character','line','season','episode','line_ID'] )
                                script_df = pd.concat([script_df,temp_df], ignore_index=True, axis=0)
                            else:
                                seas=int(int(link_text[:2]))*100
                                ep=int(link_text[3:5])
                                lineID= int((seas)+(ep))
                                temp_df = pd.DataFrame([[name,line,seas,ep,lineID]],
                                                       columns =['character','line','season','episode','line_ID'] )
                                script_df = pd.concat([script_df,temp_df], ignore_index=True, axis=0)
                    except:
                        print('SKIPPPPPPPPPPPPPED')
                        pass
                browser.back()

In [38]:
new_df=script_df
new_df[new_df['episode'] == 99]

,character,line,season,episode,line_ID


In [39]:
new_df.to_csv('./Data/Scripts.csv')